# ETH: Download Historical Data from Compound API

In [2]:
#Libraries
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from pandas.io.json import json_normalize
import pickle
import os 
import sys

In [3]:
pathAPI = os.getcwd()
pathData = os.path.join(pathAPI,'data')

In [4]:
# API URL Compound, https://compound.finance/docs/api#MarketHistoryService
urlcomp = 'https://api.compound.finance/api/v2/market_history/graph'

In [5]:
#Assets
cETH='0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5'
cSAI='0xf5dce57282a584d2746faf1593d3121fcac444dc'
cDAI='0x5d3a536e4d6dbd6114cc1ead35777bab948e3643'
cUSDC='0x39aa39c021dfbae8fac545936693ac917d5e7563'

In [6]:
#Time Frame
startT='21-06-8 00:00:00'
endT='21-06-9 00:00:00'
startT=datetime.strptime(startT, '%y-%m-%d %H:%M:%S')
endT=datetime.strptime(endT, '%y-%m-%d %H:%M:%S')

In [7]:
#To Unix time stamp
UstartT=datetime.timestamp(startT)
UendT=datetime.timestamp(endT)

In [8]:
print(UstartT)
print(UendT)

1623128400.0
1623214800.0


In [9]:
#Blocks per day
BlocksPerDay=6570

In [22]:
params = {"asset": cUSDC,
 "min_block_timestamp": 1623128400,
 "max_block_timestamp": 1623214800,
 "num_buckets": BlocksPerDay}

In [23]:
response = requests.get(urlcomp,params)
cethHD = response.json()

In [14]:
#Save Data
#f = open(os.path.join(pathData,'cETH_block_10_05_21.pickle'), 'wb')
#pickle.dump(cethHD, f,-1)          
#f.close()    

In [15]:
#Load Downloaded Data
#f = open(os.path.join(pathData,'cETH_block_10_05_21.pickle'), 'rb')
#cethHD = pickle.load(f)     
#f.close() 

In [24]:
#Retrive data
BR=json_normalize(cethHD['borrow_rates'],sep="_")
BR['rate'] = BR['rate'].astype(float)
BR.index=pd.to_datetime(BR['block_timestamp'], unit='s')
BR=BR.rate
SR=json_normalize(cethHD['supply_rates'],sep="_")
SR['rate'] = SR['rate'].astype(float)
SR.index=pd.to_datetime(SR['block_timestamp'], unit='s')
SR=SR.rate
TS=json_normalize(cethHD['total_supply_history'],sep="_")
TS['total_value'] = TS['total_value'].astype(float)
TS.index=pd.to_datetime(TS['block_timestamp'], unit='s')
TS=TS.total_value
TB=json_normalize(cethHD['total_borrows_history'],sep="_")
TB['total_value'] = TB['total_value'].astype(float)
TB.index=pd.to_datetime(TB['block_timestamp'], unit='s')
TB=TB.total_value
Xrate=json_normalize(cethHD['exchange_rates'],sep="_")
Xrate['rate'] = Xrate['rate'].astype(float)
Xrate.index=pd.to_datetime(Xrate['block_timestamp'], unit='s')
Xrate=Xrate.rate
PUSD=json_normalize(cethHD['prices_usd'],sep="_")
PUSD['price_value'] = PUSD['price_value'].astype(float)
PUSD.index=pd.to_datetime(PUSD['block_timestamp'], unit='s')
PUSD=PUSD.price_value
Uta=pd.Series((SR.values/(BR.values*(1-0.2))),index=BR.index) #Reserve Factor 20%

In [25]:
#ETH data
frame = {'Borrow Rate': BR, 'Utilization Rate': Uta,'Supply Rate': SR, 'T. Borrow': TB, 'T. Supply': TS,  'X_rate_cToken': Xrate, 'Tk.Price(USD)': PUSD }
D_ETH = pd.DataFrame(frame)
D_ETH.head()
D_ETH.to_excel('CompundH_USDCblock_8_06_21.xlsx')